## Setup

In [ ]:
!pip install -q git+https://github.com/rwightman/pytorch-image-models

In [ ]:
from timm.data.constants import IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD

import timm
import torch
import tensorflow as tf
import tensorflow_datasets as tfds 

import pickle
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

In [ ]:
BATCH_SIZE = 64 # Reduce if ResourceExhaustedError happens
AUTO = tf.data.AUTOTUNE

## Data loader

Don't execute.

In [ ]:
normalization_layer = tf.keras.layers.Normalization(mean=np.array(IMAGENET_DEFAULT_MEAN),
                                                    variance=np.array(IMAGENET_DEFAULT_STD) ** 2)

def preprocess_image(image, label):
    image = tf.cast(image, tf.float32) / 255.
    image = normalization_layer(image)
    return image, label

In [ ]:
IMG_SIZE = 256

imagenet_a = tfds.load("imagenet_a", split="test", as_supervised=True)
imagenet_a = (
    imagenet_a
    .map(lambda x, y: (tf.image.resize(x, (IMG_SIZE, IMG_SIZE)), y))
    .batch(BATCH_SIZE)
    .map(preprocess_image, num_parallel_calls=True)
    .prefetch(AUTO)
)
image_batch, label_batch = next(iter(imagenet_a))
print(image_batch.shape, label_batch.shape)

(64, 224, 224, 3) (64,)


In [ ]:
tf.reduce_max(image_batch), tf.reduce_min(image_batch)

(<tf.Tensor: shape=(), dtype=float32, numpy=2.64>,
 <tf.Tensor: shape=(), dtype=float32, numpy=-2.117904>)

## BotNet Assessment

Weight not yet available :( 

In [ ]:
# Load BoTNet models
all_botnet_models = timm.list_models("botnet*")
all_botnet_models

['botnet26t_256', 'botnet50ts_256']

In [ ]:
!gdown --id 1-jvhJaMyy-KziAuFnmt5rkoZrm5364UF

Downloading...
From: https://drive.google.com/uc?id=1-jvhJaMyy-KziAuFnmt5rkoZrm5364UF
To: /content/botnet50.pth.tar
83.8MB [00:02, 34.6MB/s]


In [ ]:
!tar xf botnet50.pth.tar

tar: This does not look like a tar archive
tar: Skipping to next header
tar: Exiting with failure status due to previous errors


In [ ]:
top_1_accs = {}
top_5_accs = {}

top_1 = tf.keras.metrics.SparseCategoricalAccuracy()
top_5 = tf.keras.metrics.SparseTopKCategoricalAccuracy(k=5)

for botnet_model_name in tqdm(all_botnet_models):
    print(f"Evaluating {botnet_model_name}")
    
    botnet_model = timm.create_model(botnet_model_name, pretrained=True)
    botnet_model.eval()
    botnet_model = botnet_model.to("cuda")
    
    all_top_1 = []
    all_top_5 = []
    
    for image_batch, label_batch in imagenet_a.as_numpy_iterator():
        with torch.no_grad():
            image_batch = torch.Tensor(label_batch).to("cuda")
            logits = botnet_model(image_batch)

        batch_accuracy_top_1 = top_1(label_batch, logits.cpu().numpy())
        batch_accuracy_top_5 = top_5(label_batch, logits.cpu().numpy())
        all_top_1.append(batch_accuracy_top_1)
        all_top_5.append(batch_accuracy_top_5)
    
    top_1_accs.update({botnet_model_name: np.mean(all_top_1)})
    top_5_accs.update({botnet_model_name: np.mean(all_top_5)})

## Other attention-infused models<sup>*</sup>

<sup>*</sup><sup>As mentioned in [`timm`](https://github.com/rwightman/pytorch-image-models)</sup>


* Bottleneck Transformer - https://arxiv.org/abs/2101.11605
* CBAM - https://arxiv.org/abs/1807.06521
* Effective Squeeze-Excitation (ESE) - https://arxiv.org/abs/1911.06667
* Efficient Channel Attention (ECA) - https://arxiv.org/abs/1910.03151
* Gather-Excite (GE) - https://arxiv.org/abs/1810.12348
* Global Context (GC) - https://arxiv.org/abs/1904.11492
* Halo - https://arxiv.org/abs/2103.12731
* Involution - https://arxiv.org/abs/2103.06255
* Lambda Layer - https://arxiv.org/abs/2102.08602
* Non-Local (NL) - https://arxiv.org/abs/1711.07971
* Squeeze-and-Excitation (SE) - https://arxiv.org/abs/1709.01507
* Selective Kernel (SK) - https://arxiv.org/abs/1903.06586
* Split (SPLAT) - https://arxiv.org/abs/2004.08955
* Shifted Window (SWIN) - https://arxiv.org/abs/2103.14030


### Common utilities

In [ ]:
def get_normalization_layer(imagenet_stats=False, scale=None, offset=None):
    if imagenet_stats:
        return tf.keras.layers.Normalization(
            mean=np.array(IMAGENET_DEFAULT_MEAN),
            variance=np.array(IMAGENET_DEFAULT_STD) ** 2
        )
    elif (scale and offset):
        return tf.keras.layers.Rescaling(
            scale=scale, offset=offset
        )
    else:
        return tf.keras.layers.Rescaling(scale=1./255)


def preprocess_image(normalization_layer):
    def f(image, label):
        if isinstance(normalization_layer, tf.keras.layers.Normalization):
            image = tf.cast(image, tf.float32) / 255.
        else:
            image = tf.cast(image, tf.float32)
        image = normalization_layer(image)
        return image, label
    return f


def get_dataset(imagenet_stats=False, resize=224, scale=None, offset=None):
    if imagenet_stats:
        norm_layer = get_normalization_layer(imagenet_stats)
    elif (scale and offset):
        norm_layer = get_normalization_layer(imagenet_stats, scale, offset)
    else:
        norm_layer = get_normalization_layer()
        
    imagenet_r = tfds.load("imagenet_r", split="test", as_supervised=True)
    imagenet_r = (
        imagenet_r
        .map(lambda x, y: (tf.image.resize(x, (resize, resize)), y))
        .batch(BATCH_SIZE)
        .map(preprocess_image(norm_layer), num_parallel_calls=True)
        .prefetch(AUTO)
    )
    return imagenet_r
    

In [ ]:
# Verify 
ds = get_dataset()
image_batch, label_batch = next(iter(ds))
print(image_batch.shape, label_batch.shape)
print(tf.reduce_max(image_batch), tf.reduce_min(image_batch))

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imagenet_r/0.1.0.incomplete6E6ELZ/imagenet_r-test.tfrecord


  0%|          | 0/30000 [00:00<?, ? examples/s]

Dataset imagenet_r downloaded and prepared to /root/tensorflow_datasets/imagenet_r/0.1.0. Subsequent calls will reuse this data.
(64, 224, 224, 3) (64,)
tf.Tensor(1.0, shape=(), dtype=float32) tf.Tensor(0.0, shape=(), dtype=float32)


In [ ]:
def eval_single_model(dataset, model_name):
    top_1 = tf.keras.metrics.SparseCategoricalAccuracy()
    top_5 = tf.keras.metrics.SparseTopKCategoricalAccuracy(k=5)

    model = timm.create_model(model_name, pretrained=True)
    model.eval()
    model = model.to("cuda")

    all_top_1 = []
    all_top_5 = []
    
    for image_batch, label_batch in dataset.as_numpy_iterator():
        with torch.no_grad():
            image_batch = torch.Tensor(image_batch).to("cuda")
            image_batch = image_batch.permute(0, 3, 1, 2)
            logits = model(image_batch)

        batch_accuracy_top_1 = top_1(label_batch, logits.cpu().numpy())
        batch_accuracy_top_5 = top_5(label_batch, logits.cpu().numpy())
        all_top_1.append(batch_accuracy_top_1)
        all_top_5.append(batch_accuracy_top_5)

    return np.mean(all_top_1), np.mean(all_top_5)

### Global Context

In [ ]:
all_gc_models = timm.list_models("*gc*")
all_gc_models

['gc_efficientnetv2_rw_t', 'gcresnet26ts', 'gcresnet50t', 'gcresnext26ts']

In [ ]:
# EfficientNetV2: [-1, 1]
# Reference: https://github.com/google/automl/blob/master/efficientnetv2/infer.py
dataset = get_dataset(imagenet_stats=False, scale=1./127.5, offset=-1)
image_batch, label_batch = next(iter(dataset))
print(image_batch.shape, label_batch.shape)
print(tf.reduce_max(image_batch), tf.reduce_min(image_batch))

(64, 224, 224, 3) (64,)
tf.Tensor(1.0, shape=(), dtype=float32) tf.Tensor(-1.0, shape=(), dtype=float32)


In [ ]:
top_1_accs = {}
top_5_accs = {}

# Only single model is available as pre-trained.
print(f"Evaluating {all_gc_models[0]}")

mean_top_1, mean_top_5 = eval_single_model(dataset, all_gc_models[0])

top_1_accs.update({all_gc_models[0]: np.mean(mean_top_1)})
top_5_accs.update({all_gc_models[0]: np.mean(mean_top_5)})

Evaluating gc_efficientnetv2_rw_t


Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/gc_efficientnetv2_rw_t_agc-927a0bde.pth" to /root/.cache/torch/hub/checkpoints/gc_efficientnetv2_rw_t_agc-927a0bde.pth
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  


In [ ]:
top_1_accs[all_gc_models[0]], top_5_accs[all_gc_models[0]]

(0.32701343, 0.46182257)

### Gather-Excite (GE)

In [ ]:
dataset = get_dataset(imagenet_stats=True)
image_batch, label_batch = next(iter(dataset))
print(image_batch.shape, label_batch.shape)
print(tf.reduce_max(image_batch), tf.reduce_min(image_batch))

(64, 224, 224, 3) (64,)
tf.Tensor(2.64, shape=(), dtype=float32) tf.Tensor(-2.117904, shape=(), dtype=float32)


In [ ]:
all_ge_models = timm.list_models("ge*")
all_ge_models

['geresnet50t', 'gernet_l', 'gernet_m', 'gernet_s']

First model is not available as ImageNet-1k pretrained.

In [ ]:
top_1_accs = {}
top_5_accs = {}


for ge_model_name in tqdm(all_ge_models[1:]):
    print(f"Evaluating {ge_model_name}")
    
    mean_top_1, mean_top_5 = eval_single_model(dataset, ge_model_name)

    top_1_accs.update({ge_model_name: mean_top_1})
    top_5_accs.update({ge_model_name: mean_top_5})
    

  0%|          | 0/3 [00:00<?, ?it/s]

Evaluating gernet_l


Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-ger-weights/gernet_l-f31e2e8d.pth" to /root/.cache/torch/hub/checkpoints/gernet_l-f31e2e8d.pth


Evaluating gernet_m


Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-ger-weights/gernet_m-0873c53a.pth" to /root/.cache/torch/hub/checkpoints/gernet_m-0873c53a.pth


Evaluating gernet_s


Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-ger-weights/gernet_s-756b4751.pth" to /root/.cache/torch/hub/checkpoints/gernet_s-756b4751.pth


In [ ]:
top_1_accs, top_5_accs

({'gernet_l': 0.29905242, 'gernet_m': 0.29345, 'gernet_s': 0.2496156},
 {'gernet_l': 0.43354014, 'gernet_m': 0.4459289, 'gernet_s': 0.39041907})

### Selective kernel

In [ ]:
all_sk_models = timm.list_models("sk*")
all_sk_models

['skresnet18', 'skresnet34', 'skresnet50', 'skresnet50d', 'skresnext50_32x4d']

In [ ]:
top_1_accs = {}
top_5_accs = {}


for sk_model_name in tqdm(all_sk_models):
    if sk_model_name not in ["skresnet50", "skresnet50d"]:
        print(f"Evaluating {sk_model_name}")
        
        mean_top_1, mean_top_5 = eval_single_model(dataset, sk_model_name)

        top_1_accs.update({sk_model_name: mean_top_1})
        top_5_accs.update({sk_model_name: mean_top_5})

top_1_accs, top_5_accs

  0%|          | 0/5 [00:00<?, ?it/s]

Evaluating skresnet18


Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/skresnet18_ra-4eec2804.pth" to /root/.cache/torch/hub/checkpoints/skresnet18_ra-4eec2804.pth
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Evaluating skresnet34


Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/skresnet34_ra-bdc0ccde.pth" to /root/.cache/torch/hub/checkpoints/skresnet34_ra-bdc0ccde.pth


Evaluating skresnext50_32x4d


Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/skresnext50_ra-f40e40bf.pth" to /root/.cache/torch/hub/checkpoints/skresnext50_ra-f40e40bf.pth


({'skresnet18': 0.2295351,
  'skresnet34': 0.2677625,
  'skresnext50_32x4d': 0.283357},
 {'skresnet18': 0.36674884,
  'skresnet34': 0.40519035,
  'skresnext50_32x4d': 0.42141196})